In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ORRFA import *
import interpretableai
from interpretableai import iai
import pandas as pd
import json
from HR import *

  HTTP.Exceptions.RequestError(HTTP.Messages.Request:
"""
GET /versions.json HTTP/1.1
Host: docs.interpretable.ai
Accept: */*
User-Agent: HTTP.jl/1.8.1
Content-Length: 0
Accept-Encoding: gzip

""", EOFError())


In [2]:
dataset = pd.read_csv("/Users/ryanlucas/Desktop/kc_house_data.csv")

In [3]:
dataset

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [4]:
train_idx = np.arange(0, int(0.2*len(dataset)))
test_idx = np.arange(int(0.2*len(dataset)), int(0.3*len(dataset)))

features = dataset.loc[train_idx, ["floors", "bedrooms",
    "bathrooms",
    "sqft_above",
    "sqft_basement",
    "yr_built",
    "yr_renovated",
    "sqft_living",
    "sqft_lot",
    "lat",
    "long",
    "sqft_living15",
    "sqft_lot15"]]

diagnosis = dataset.loc[train_idx, "price"]

orig_features_size = features.copy().shape[1]

In [5]:
features_test = dataset.loc[test_idx, ["floors", "bedrooms",
    "bathrooms",
    "sqft_above",
    "sqft_basement",
    "yr_built",
    "yr_renovated",
    "sqft_living",
    "sqft_lot",
    "lat",
    "long",
    "sqft_living15",
    "sqft_lot15"]]
diagnosis_test = dataset.loc[test_idx, 'price']

In [6]:
diagnosis

0       221900.0
1       538000.0
2       180000.0
3       604000.0
4       510000.0
          ...   
4317    250000.0
4318    402500.0
4319    432500.0
4320    485000.0
4321    560000.0
Name: price, Length: 4322, dtype: float64

In [7]:
print(len(dataset))
print(len(features))
print(len(features_test))

21613
4322
2161


In [8]:
features

,floors,bedrooms,bathrooms,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living,sqft_lot,lat,long,sqft_living15,sqft_lot15
0,1.0,3,1.00,1180,0,1955,0,1180,5650,47.5112,-122.257,1340,5650
1,2.0,3,2.25,2170,400,1951,1991,2570,7242,47.7210,-122.319,1690,7639
2,1.0,2,1.00,770,0,1933,0,770,10000,47.7379,-122.233,2720,8062
3,1.0,4,3.00,1050,910,1965,0,1960,5000,47.5208,-122.393,1360,5000
4,1.0,3,2.00,1680,0,1987,0,1680,8080,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,1.0,3,1.00,1060,0,1960,0,1060,52272,47.3600,-122.178,1900,5971
4318,1.5,3,1.00,1290,0,1926,0,1290,4000,47.5462,-122.277,1160,5040
4319,1.0,3,2.50,1600,790,1978,0,2390,6435,47.7260,-122.194,2020,7300
4320,1.0,5,1.75,1380,1080,1972,0,2460,14100,47.7452,-122.224,2028,11078


In [9]:
def gen_features(sub_paths, data, features, orig_feature_size):
    new_features = features.copy()
    for path in sub_paths:
        statement = gen_statement(path, data, features)
        if statement != '(0 < 0.0)' and statement != '(0 >= 0.0)':

            try:
                index_feature = features.loc[eval(statement)].index


                new_feature = [1 if i in index_feature else 0 for i in range(len(features))]

                if sum(new_feature) >= 1:
                    new_features[statement] = new_feature

            except:
                print(statement)
    return new_features.iloc[:, orig_feature_size:]

In [10]:
np.random.seed(1)
n = 5
rules = {}
for i in range(1,15):

    #################
    sampled_features = features.copy().sample(n=int(n),axis='columns')

    OCT_H = iai.GridSearch(
          iai.OptimalTreeRegressor(
              max_depth = 3, cp = 0.00000000001))

    OCT_H.fit(sampled_features, diagnosis)
    OCT_H.write_json(f"tre_{i}.json")

    ######################
    f = open(f"tre_{i}.json")
    data = json.load(f)

    paths = gen_paths(data)

    rule_features_new = gen_features(
            paths, data, features, orig_features_size)

    rules[f"tre_{i}"] = rule_features_new

    print(f"Tree: {i}")


[ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.


KeyboardInterrupt: 

In [ ]:
OCT_H.get_learner()

In [ ]:
features.columns

In [ ]:
data['lnr']['tree_']['nodes'][11]

In [ ]:
features.columns

In [ ]:
big_df = pd.DataFrame()
for tree in rules.keys():

    big_df = pd.concat([big_df, rules[tree]],axis =1)


In [ ]:
df = big_df.loc[:,~big_df.columns.duplicated()].copy()

In [ ]:
df

In [ ]:
for column in df.columns[1:]:
    if df[column].sum() == len(df):
        del df[column]


In [ ]:
df

In [ ]:
features_for_robust_reg = df

In [ ]:
rule_features = features_for_robust_reg.copy()
features_for_robust_reg = pd.concat([features, features_for_robust_reg], axis = 1)

In [ ]:
features_for_robust_reg.sum()

In [ ]:
features_for_robust_reg.insert(loc = 0, column = 'Intercept', value = [1 for i in range(len(features_for_robust_reg))])

In [ ]:
features_for_robust_reg

In [ ]:
features_for_robust_reg

In [ ]:
model = HolisticRobust(
                       α = 0,
                       ϵ = 0,
                       r = 0.05,
                       classifier = "Linear",
                       learning_approach = "HR") # Could be ERM either

In [ ]:
θ, obj_value = model.fit(ξ = (np.matrix(features_for_robust_reg), np.array(diagnosis)))

In [ ]:
predictions = np.multiply(θ, features_for_robust_reg
                          ).sum(axis = 1)

In [ ]:
diagnosis.plot()

In [ ]:
(diagnosis - predictions).describe()

In [ ]:
SSE_model = sum((diagnosis - predictions)**2)
SSE_mean = sum((diagnosis - diagnosis.mean())**2)

1 - SSE_model/SSE_mean

In [ ]:
features

In [ ]:
rule_features

In [ ]:
features = features_test

for rule in rule_features.columns:

    index_feature = features.loc[eval(rule)].index

    new_feature = [1 if i in index_feature else 0 for i in range(len(features))]

    if sum(new_feature) >= 1:
        features[rule] = new_feature


In [ ]:
new_features = features_test

In [ ]:
new_features.insert(loc = 0, column = 'Intercept', value = [1 for i in range(len(new_features))])

In [ ]:
predictions_test = lasso_trees.predict(features_test)